<h1 align='center'>Battle of neighborhood-Bangalore </h1>

## Introduction

Bangalore is one of the metropolitan cities in India and the capital of Karnataka state. Bangalore is third most populated city in India.  So many multinational companies are having their offices in Bangalore. Especially major Information technology companies have presence in Bangalore, often called as Silicon Valley of India or IT capital of India. In one of the article published in 2018 mentioned that Bangalore has one of the most highly educated workforces in the world. Bangalore also home to many educational and research institutions, such as Indian Institute of Science (IISc), Indian Institute of Management (Bangalore) (IIMB), International Institute of Information Technology, Bangalore (IIITB), National Institute of Fashion Technology, Bangalore, National Institute of Design, Bangalore (NID R&D Campus), National Law School of India University (NLSIU) and National Institute of Mental Health and Neurosciences (NIMHANS). Numerous state-owned aerospace and defense organizations, such as Bharat Electronics, Hindustan Aeronautics and National Aerospace Laboratories are located in the city. The city also houses the Kannada film industry also called Sandalwood.
    Because of these above distinct features of the city many people from other states of India as well as from other countries will come and work in Bangalore. From the people who are coming from other place want to explore the neighborhoods of Bangalore for their residence. Also, because of distinct culture of people present there provides the opportunity of starting a new business in a locality.  
    I would like to explore the neighborhoods of Bangalore, India. Also, I am interested in segmenting the neighborhoods into clusters. If someone relocating to Bangalore required to find a suitable place to stay. It will help me in identifying which neighborhoods are good for housing etc. As a new person to Bangalore, it will help in identifying good location. The good location like nearby schools, colleges, hospitals etc.


## Data Section:

We need to get the neighborhoods of the city Bangalore. Sign up into the foursquare api account. Get the credentials and secret of the developer api account in foursquare. Using the foursquare api, get the latitude and longitudes of all the neighborhoods of Bangalore. A request has to be send to foursquare. Results are in the format of json. Observing the format of the data, required information need to be extracted. Then we need to analyze the data finally use clustering algorithm for segmentation.  

## Methodology

### Import necessary libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


### Get the geographical coordinates of Bangalore using nominatim 

In [4]:
address = 'Bangalore'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


### Foursquare data

In [5]:
CLIENT_ID = 'NS0CYTNGCRSMJRSL451UKJYIGCXYSSCZGOBT5OV1WLXWGFA1' # your Foursquare ID
CLIENT_SECRET = 'BY2QFIQLD5F041LR4ZJZIHBY134GKG4ECR244TX2LOXE1LP1' # your Foursquare Secret
VERSION = '20200222' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NS0CYTNGCRSMJRSL451UKJYIGCXYSSCZGOBT5OV1WLXWGFA1
CLIENT_SECRET:BY2QFIQLD5F041LR4ZJZIHBY134GKG4ECR244TX2LOXE1LP1


<b>Create an instance to foursquare api

In [7]:
LIMIT=1000
radius=15000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

<b> Request the instance

In [8]:
results = requests.get(url).json()
json_normalize(results)

,meta.code,meta.requestId,response.groups,response.headerFullLocation,response.headerLocation,response.headerLocationGranularity,response.suggestedBounds.ne.lat,response.suggestedBounds.ne.lng,response.suggestedBounds.sw.lat,response.suggestedBounds.sw.lng,response.suggestedFilters.filters,response.suggestedFilters.header,response.totalResults
0,200,5e5024516001fe001b565271,"[{'type': 'Recommended Places', 'name': 'recom...",Bangalore,Bangalore,city,13.11412,77.729581,12.84412,77.453019,"[{'name': 'Open now', 'key': 'openNow'}]",Tap to show:,238


Create a function to extract category type

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Data Analysis

In [10]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print(nearby_venues.shape)
nearby_venues.head()

(100, 4)


,name,categories,lat,lng
0,Cubbon Park,Park,12.977042,77.595277
1,JW Marriott Hotel Bengaluru,Hotel,12.972362,77.595051
2,Taj West End,Hotel,12.984572,77.584893
3,UB City,Shopping Mall,12.971709,77.595905
4,Toscano,Italian Restaurant,12.971980,77.596066


In [11]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [12]:
nearby_venues['name'].unique()

array(['Cubbon Park', 'JW Marriott Hotel Bengaluru', 'Taj West End',
       'UB City', 'Toscano', 'Infinitea', 'Hard Rock Cafe Bengaluru',
       'Truffles - Ice & Spice', 'Smoke House Deli', 'Corner House',
       'M. Chinnaswamy Stadium', 'Harima', 'Skyye', 'Café Noir',
       'M.G Road Boulevard', 'Shiro', 'ITC Gardenia',
       'Ujwal Bar & Restaurant', 'The Ritz-Carlton', 'Blossom Book House',
       'J W Kitchen', 'Spice Terrace', 'Airlines Hotel', 'Millers 46',
       'Arbor Brewing Company', 'Samarkand', 'ITC Windsor',
       'The Biere Club', 'MTR 1924', 'The Oberoi', 'Starbucks',
       'Bangalore Turf Club', 'Chitra Kala Parishad',
       'Bangalore Golf Club', 'Church Street Social',
       'Shangri-La Hotel, Bengaluru', 'Matteo', 'Toast and Tonic',
       'Shri Sagar (Formerly CTR)', 'Plan B', 'The Permit Room',
       'Brahmins Coffee Bar', 'Asha Sweet Center', 'Kebabs & Kurries',
       'Kentacky Chicken Corner', 'Foodhall', 'The Only Place',
       'Vivanta by Taj', "Wa

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
Bangalore_venues = getNearbyVenues(names=nearby_venues['name'],
                                   latitudes=nearby_venues['lat'],
                                   longitudes=nearby_venues['lng']
                                  )


Cubbon Park
JW Marriott Hotel Bengaluru
Taj West End
UB City
Toscano
Infinitea
Hard Rock Cafe Bengaluru
Truffles - Ice & Spice
Smoke House Deli
Corner House
M. Chinnaswamy Stadium
Harima
Skyye
Café Noir
M.G Road Boulevard
Shiro
ITC Gardenia
Ujwal Bar & Restaurant
Corner House
The Ritz-Carlton
Blossom Book House
J W Kitchen
Spice Terrace
Airlines Hotel
Millers 46
Arbor Brewing Company
Samarkand
ITC Windsor
The Biere Club
MTR 1924
The Oberoi
Starbucks
Bangalore Turf Club
Chitra Kala Parishad
Bangalore Golf Club
Church Street Social
Shangri-La Hotel, Bengaluru
Matteo
Toast and Tonic
Shri Sagar (Formerly CTR)
Plan B
The Permit Room
Brahmins Coffee Bar
Asha Sweet Center
Kebabs & Kurries
Kentacky Chicken Corner
Foodhall
The Only Place
Vivanta by Taj
Watson's
Conrad Bengaluru
Desserted
Nagarjuna
Mangalore Pearl - Seafood Restaurant
Marks & Spencer
O.G. Variar & Sons
Mavalli Tiffin Room (MTR)
Lakshmi Nataraj Refreshments
Mahesh Lunch Home
Bangalore Brew Works
Richard's Park
Corner House
The Ra

### Print the nearby venues

In [17]:
print(Bangalore_venues.shape)
Bangalore_venues.head()

(4022, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cubbon Park,12.977042,77.595277,Cubbon Park,12.977042,77.595277,Park
1,Cubbon Park,12.977042,77.595277,M. Chinnaswamy Stadium,12.978144,77.599223,Cricket Ground
2,Cubbon Park,12.977042,77.595277,"Shiro, UB City",12.976748,77.599066,Lounge
3,Cubbon Park,12.977042,77.595277,Venkatappa Art Gallery,12.974575,77.595792,Art Gallery
4,Cubbon Park,12.977042,77.595277,Vidhana Soudha,12.979027,77.591881,Capitol Building


The following cell gives the count in each neighborhood

In [18]:
Bangalore_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adarsh Hamilton,44,44,44,44,44,44
Affinity Gym,15,15,15,15,15,15
Airlines Hotel,45,45,45,45,45,45
Arbor Brewing Company,58,58,58,58,58,58
Art of Delight,79,79,79,79,79,79
Asha Sweet Center,18,18,18,18,18,18
Bangalore Brew Works,55,55,55,55,55,55
Bangalore Golf Club,25,25,25,25,25,25
Bangalore Turf Club,19,19,19,19,19,19


In [19]:
print('There are {} uniques categories.'.format(len(Bangalore_venues['Venue Category'].unique())))

There are 182 uniques categories.


In [20]:
# one hot encoding
Bangalore_onehot = pd.get_dummies(Bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Bangalore_onehot['Neighborhood'] = Bangalore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Bangalore_onehot.columns[-1]] + list(Bangalore_onehot.columns[:-1])
Bangalore_onehot = Bangalore_onehot[fixed_columns]

Bangalore_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bed & Breakfast,Bengali Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Café,Camera Store,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multiplex,Museum,Music Store,Music Venue,Nightclub,North Indian Restaurant,Office,Optical Shop,Other Nightlife,Outlet Store,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Persian Restaurant,Pharmacy,Photography Studio,Pizza Place,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Real Estate Office,Residential Building (Apartment / Condo),Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Toy / Game Store,Track Stadium,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Cubbon Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Cubbon Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Cubbon Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Cubbon Park,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [21]:
Bangalore_onehot.shape

(4022, 183)

In [22]:
Bangalore_grouped = Bangalore_onehot.groupby('Neighborhood').mean().reset_index()
Bangalore_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bed & Breakfast,Bengali Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Café,Camera Store,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multiplex,Museum,Music Store,Music Venue,Nightclub,North Indian Restaurant,Office,Optical Shop,Other Nightlife,Outlet Store,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Persian Restaurant,Pharmacy,Photography Studio,Pizza Place,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Real Estate Office,Residential Building (Apartment / Condo),Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Toy / Game Store,Track Stadium,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Adarsh Hamilton,0.000000,0.00,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.045455,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.022727,0.000000,0.000000,0.00000,0.000000,0.000000,0.045455,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.045455,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.022727,0.000000,0.022727,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.022727,0.022727,0.000000,0.000000,0.113636,0.000000,0.000000,0.000000,0.000000,0.136364,0.000000,0.000000,0.000000,0.022727,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [23]:
Bangalore_grouped.shape

(94, 183)

### Get the top venues

In [24]:
num_top_venues = 5

for hood in Bangalore_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Bangalore_grouped[Bangalore_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adarsh Hamilton----
                venue  freq
0   Indian Restaurant  0.14
1               Hotel  0.11
2  Chinese Restaurant  0.09
3                Café  0.05
4         Pizza Place  0.05


----Affinity Gym----
               venue  freq
0  Indian Restaurant  0.20
1        Coffee Shop  0.20
2   Department Store  0.13
3               Café  0.13
4      Women's Store  0.07


----Airlines Hotel----
                venue  freq
0  Mexican Restaurant  0.07
1   Indian Restaurant  0.07
2  Italian Restaurant  0.07
3      Breakfast Spot  0.04
4              Lounge  0.04


----Arbor Brewing Company----
                venue  freq
0   Indian Restaurant  0.10
1                Café  0.09
2               Hotel  0.09
3          Donut Shop  0.07
4  Chinese Restaurant  0.05


----Art of Delight----
                venue  freq
0                Café  0.13
1   Indian Restaurant  0.11
2                 Pub  0.08
3  Chinese Restaurant  0.05
4         Pizza Place  0.04


----Asha Sweet Center----
         

### Return most common venues

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### The following cell evaluates the top ten venues in each neighborhood

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Bangalore_grouped['Neighborhood']

for ind in np.arange(Bangalore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Bangalore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Hamilton,Indian Restaurant,Hotel,Chinese Restaurant,Pizza Place,Café,Bar,Coffee Shop,Sushi Restaurant,Breakfast Spot,Pub
1,Affinity Gym,Indian Restaurant,Coffee Shop,Department Store,Café,Seafood Restaurant,Women's Store,Gym,Plaza,Ice Cream Shop,Asian Restaurant
2,Airlines Hotel,Indian Restaurant,Mexican Restaurant,Italian Restaurant,Lounge,Electronics Store,Breakfast Spot,Ice Cream Shop,Furniture / Home Store,Clothing Store,Brewery
3,Arbor Brewing Company,Indian Restaurant,Hotel,Café,Donut Shop,Pub,Chinese Restaurant,Clothing Store,Electronics Store,Brewery,Sandwich Place
4,Art of Delight,Café,Indian Restaurant,Pub,Chinese Restaurant,Bar,Hotel,Donut Shop,Coffee Shop,Clothing Store,Pizza Place


The above data need to cluster into different groups

### Clustering

In [28]:
# set number of clusters
kclusters = 5

Bangalore_grouped_clustering = Bangalore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Bangalore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 2, 2, 0, 1, 0, 2, 0, 0])

In [29]:
Bangalore_data=nearby_venues
Bangalore_data.rename(columns={'name':'Neighborhood'})

,Neighborhood,categories,lat,lng
0,Cubbon Park,Park,12.977042,77.595277
1,JW Marriott Hotel Bengaluru,Hotel,12.972362,77.595051
2,Taj West End,Hotel,12.984572,77.584893
3,UB City,Shopping Mall,12.971709,77.595905
4,Toscano,Italian Restaurant,12.971980,77.596066
5,Infinitea,Tea Room,12.987157,77.594835
6,Hard Rock Cafe Bengaluru,American Restaurant,12.976389,77.601468
7,Truffles - Ice & Spice,Burger Joint,12.971802,77.601031
8,Smoke House Deli,Deli / Bodega,12.971656,77.598254
9,Corner House,Ice Cream Shop,12.972979,77.599971


## Results

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Bangalore_merged = Bangalore_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Bangalore_merged = Bangalore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='name')

Bangalore_merged.head() # check the last columns!

,name,categories,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cubbon Park,Park,12.977042,77.595277,2,Tennis Stadium,Capitol Building,Plaza,Cricket Ground,Park,Fast Food Restaurant,Art Gallery,Science Museum,Museum,Japanese Restaurant
1,JW Marriott Hotel Bengaluru,Hotel,12.972362,77.595051,2,Italian Restaurant,Lounge,Café,Mexican Restaurant,Hotel,Electronics Store,Coffee Shop,Clothing Store,Restaurant,Cocktail Bar
2,Taj West End,Hotel,12.984572,77.584893,0,Indian Restaurant,Hotel,Snack Place,Juice Bar,Gym,Garden,Office,Park,Hotel Pool,Coffee Shop
3,UB City,Shopping Mall,12.971709,77.595905,2,Italian Restaurant,Mexican Restaurant,Café,Lounge,Hotel,Electronics Store,Coffee Shop,Restaurant,Clothing Store,Rajasthani Restaurant
4,Toscano,Italian Restaurant,12.971980,77.596066,2,Italian Restaurant,Café,Mexican Restaurant,Lounge,Electronics Store,Hotel,Coffee Shop,Clothing Store,Restaurant,Cocktail Bar


### Visualize in map

In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Bangalore_merged['lat'], Bangalore_merged['lng'], Bangalore_merged['name'], Bangalore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

In [50]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
map_clusters

<b>Cluster 1

In [33]:
Bangalore_merged.loc[Bangalore_merged['Cluster Labels'] == 0, Bangalore_merged.columns[[1] + list(range(5, Bangalore_merged.shape[1]))]]

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Hotel,Indian Restaurant,Hotel,Snack Place,Juice Bar,Gym,Garden,Office,Park,Hotel Pool,Coffee Shop
5,Tea Room,Indian Restaurant,Chinese Restaurant,Hotel,Coffee Shop,Pizza Place,Italian Restaurant,Café,Optical Shop,Hookah Bar,Fast Food Restaurant
6,American Restaurant,Indian Restaurant,Lounge,Ice Cream Shop,Breakfast Spot,Steakhouse,Movie Theater,Music Store,Cricket Ground,Pizza Place,Pub
7,Burger Joint,Indian Restaurant,Ice Cream Shop,Brewery,Breakfast Spot,Bubble Tea Shop,Steakhouse,Café,Restaurant,Burger Joint,Deli / Bodega
9,Ice Cream Shop,Indian Restaurant,Ice Cream Shop,Italian Restaurant,Lounge,Hotel,Chinese Restaurant,Mexican Restaurant,Pub,Clothing Store,Cocktail Bar
10,Cricket Ground,Indian Restaurant,Lounge,Bar,Cricket Ground,Plaza,Pub,Chinese Restaurant,Fast Food Restaurant,Sandwich Place,Movie Theater
14,Plaza,Indian Restaurant,Lounge,Café,Pub,Coffee Shop,Bubble Tea Shop,Burger Joint,Road,Clothing Store,Pizza Place
17,Indian Restaurant,Indian Restaurant,Chinese Restaurant,Italian Restaurant,Coffee Shop,Snack Place,Steakhouse,Juice Bar,Fast Food Restaurant,Electronics Store,Music Venue
18,Ice Cream Shop,Indian Restaurant,Ice Cream Shop,Italian Restaurant,Lounge,Hotel,Chinese Restaurant,Mexican Restaurant,Pub,Clothing Store,Cocktail Bar
20,Bookstore,Pub,Indian Restaurant,Café,Bookstore,Coffee Shop,Electronics Store,Lounge,Fried Chicken Joint,Fast Food Restaurant,Music Store


<b>Cluster 2

In [35]:
Bangalore_merged.loc[Bangalore_merged['Cluster Labels'] == 1, Bangalore_merged.columns[[1] + list(range(5, Bangalore_merged.shape[1]))]]

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Indian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Bar,Jewelry Store,Donut Shop,Department Store,Pizza Place,Café,Breakfast Spot,Snack Place
42,Breakfast Spot,Indian Restaurant,Fast Food Restaurant,Café,Print Shop,Department Store,Coffee Shop,Breakfast Spot,Pool,Flea Market,Hookah Bar
43,Snack Place,Indian Restaurant,Vegetarian / Vegan Restaurant,Men's Store,Department Store,Clothing Store,Donut Shop,Café,Flea Market,Breakfast Spot,Pizza Place
68,Bakery,Indian Restaurant,Bakery,Pharmacy,Café,Fast Food Restaurant,Snack Place,Vegetarian / Vegan Restaurant,Hookah Bar,Dumpling Restaurant,Financial or Legal Service
69,Indian Restaurant,Indian Restaurant,Paper / Office Supplies Store,Women's Store,American Restaurant,Cupcake Shop,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Flea Market
82,Ice Cream Shop,Indian Restaurant,Asian Restaurant,Fast Food Restaurant,Ice Cream Shop,Andhra Restaurant,Event Space,Food Truck,Food Court,Food & Drink Shop,Flower Shop
85,Indian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Men's Store,Department Store,Clothing Store,Donut Shop,Café,Flea Market,Breakfast Spot,Pizza Place
89,Indian Restaurant,Indian Restaurant,Fast Food Restaurant,Breakfast Spot,Park,Sandwich Place,Plaza,Hockey Arena,Flea Market,Hospital,Farmers Market
97,Indian Restaurant,Indian Restaurant,Fast Food Restaurant,Sandwich Place,Pizza Place,Ice Cream Shop,Park,Asian Restaurant,Hookah Bar,Bakery,Juice Bar


<b> Cluster 3

In [36]:
Bangalore_merged.loc[Bangalore_merged['Cluster Labels'] == 2, Bangalore_merged.columns[[1] + list(range(5, Bangalore_merged.shape[1]))]]

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Park,Tennis Stadium,Capitol Building,Plaza,Cricket Ground,Park,Fast Food Restaurant,Art Gallery,Science Museum,Museum,Japanese Restaurant
1,Hotel,Italian Restaurant,Lounge,Café,Mexican Restaurant,Hotel,Electronics Store,Coffee Shop,Clothing Store,Restaurant,Cocktail Bar
3,Shopping Mall,Italian Restaurant,Mexican Restaurant,Café,Lounge,Hotel,Electronics Store,Coffee Shop,Restaurant,Clothing Store,Rajasthani Restaurant
4,Italian Restaurant,Italian Restaurant,Café,Mexican Restaurant,Lounge,Electronics Store,Hotel,Coffee Shop,Clothing Store,Restaurant,Cocktail Bar
8,Deli / Bodega,Italian Restaurant,Mexican Restaurant,Lounge,Indian Restaurant,Café,Coffee Shop,Electronics Store,Breakfast Spot,Ice Cream Shop,Hotel
11,Sushi Restaurant,Indian Restaurant,Café,Hotel,Bar,Pub,Ice Cream Shop,Restaurant,Chinese Restaurant,Coffee Shop,Motorcycle Shop
12,Lounge,Italian Restaurant,Mexican Restaurant,Lounge,Café,Electronics Store,Pub,Restaurant,Coffee Shop,Clothing Store,Hotel
13,French Restaurant,Italian Restaurant,Café,Mexican Restaurant,Lounge,Electronics Store,Hotel,Coffee Shop,Clothing Store,Restaurant,Cocktail Bar
15,Japanese Restaurant,Italian Restaurant,Mexican Restaurant,Lounge,Café,Pub,Restaurant,Electronics Store,Hotel,Clothing Store,Coffee Shop
16,Hotel,Indian Restaurant,Hotel,Café,Sushi Restaurant,Chinese Restaurant,Wine Bar,Parsi Restaurant,Seafood Restaurant,Restaurant,Cocktail Bar


<b>Cluster 4

In [37]:
Bangalore_merged.loc[Bangalore_merged['Cluster Labels'] == 3, Bangalore_merged.columns[[1] + list(range(5, Bangalore_merged.shape[1]))]]

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
87,Restaurant,Café,Indian Restaurant,Restaurant,Tea Room,English Restaurant,Food & Drink Shop,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant


<b>Cluster 5

In [39]:
Bangalore_merged.loc[Bangalore_merged['Cluster Labels'] == 4, Bangalore_merged.columns[[1] + list(range(5, Bangalore_merged.shape[1]))]]

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,Botanical Garden,Botanical Garden,Flower Shop,Park,Scenic Lookout,Yoga Studio,English Restaurant,Food & Drink Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant


## Discussion

From the cluster analysis we can infer that cluster 2 and 3 are having many venues compared with other clusters

## Conclusion

For choosing a better location for housing it would be great to choose location in cluster 2 and 3